# Style Transfer
이번 과제에서는 ["Image Style Transfer Using Convolutional Neural Networks" (Gatys et al., CVPR 2015)](http://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf).
에서 제시된 스타일 트랜스퍼 기법을 구현하는 것을 목표로 합니다.

일반적인 아이디어는 두 개의 이미지를 사용해, 하나의 이미지는 콘텐츠를 반영하고 다른 하나의 이미지는 예술적 “스타일”을 반영하는 새로운 이미지를 생성하는 것을 목표로 합니다.<br>
우리는 먼저 깊은 신경망의 특징 공간에서 각각의 이미지 콘텐츠와 스타일이 일치하도록 하는 손실 함수를 정식화한 뒤, 이미지 자체의 픽셀에 대해 경사하강을 수행합니다.<br>

특징 추출기로 사용하는 딥 네트워크는 ImageNet으로 학습된 작은 모델인 SqueezeNet
이다.<br> 어떤 네트워크든 사용할 수 있지만, 우리는 작은 크기와 효율성 때문에 SqueezeNet을 선택하여 진행합니다.

다음은 이 노트북을 통해 최종적으로 만들 수 있는 이미지의 예시입니다:
![caption](example_styletransfer.png)


## Setup

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import PIL

import numpy as np

from imageio import imread
from PIL import Image
from collections import namedtuple
import matplotlib.pyplot as plt

from utils.image_utils import SQUEEZENET_MEAN, SQUEEZENET_STD
%matplotlib inline

과제의 이 부분에서는 CIFAR-10 데이터가 아니라 실제 JPEG 이미지를 다루기 때문에, 이미지를 처리하기 위한 몇 가지 헬퍼 함수를 제공합니다.

In [ ]:
def preprocess(img, size=512):
    transform = T.Compose([
        T.Resize(size),
        T.ToTensor(),
        T.Normalize(mean=SQUEEZENET_MEAN.tolist(),
                    std=SQUEEZENET_STD.tolist()),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(img)

def deprocess(img):
    transform = T.Compose([
        T.Lambda(lambda x: x[0]),
        T.Normalize(mean=[0, 0, 0], std=[1.0 / s for s in SQUEEZENET_STD.tolist()]),
        T.Normalize(mean=[-m for m in SQUEEZENET_MEAN.tolist()], std=[1, 1, 1]),
        T.Lambda(rescale),
        T.ToPILImage(),
    ])
    return transform(img)

def rescale(x):
    low, high = x.min(), x.max()
    x_rescaled = (x - low) / (high - low)
    return x_rescaled

def rel_error(x,y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def features_from_img(imgpath, imgsize):
    img = preprocess(PIL.Image.open(imgpath), size=imgsize)
    img_var = img.type(dtype)
    return extract_features(img_var, cnn), img_var

# Older versions of scipy.misc.imresize yield different results
# from newer versions, so we check to make sure scipy is up to date.
def check_scipy():
    import scipy
    vnum = int(scipy.__version__.split('.')[1])
    major_vnum = int(scipy.__version__.split('.')[0])
    
    assert vnum >= 16 or major_vnum >= 1, "You must install SciPy >= 0.16.0 to complete this notebook."

check_scipy()

answers = dict(np.load('style-transfer-checks.npz'))


지난 과제에서처럼, CPU 또는 GPU 중 하나를 선택하기 위해 dtype을 설정해야 한다.

In [ ]:
dtype = torch.FloatTensor
# Uncomment out the following line if you're on a machine with a GPU set up for PyTorch!
#dtype = torch.cuda.FloatTensor 

In [ ]:
# Load the pre-trained SqueezeNet model.
cnn = torchvision.models.squeezenet1_1(pretrained=True).features
cnn.type(dtype)

# We don't want to train the model any further, so we don't want PyTorch to waste computation 
# computing gradients on parameters we're never going to update.
for param in cnn.parameters():
    param.requires_grad = False

# We provide this helper code which takes an image, a model (cnn), and returns a list of
# feature maps, one per layer.
def extract_features(x, cnn):
    """
    Use the CNN to extract features from the input image x.
    
    Inputs:
    - x: A PyTorch Tensor of shape (N, C, H, W) holding a minibatch of images that
      will be fed to the CNN.
    - cnn: A PyTorch model that we will use to extract features.
    
    Returns:
    - features: A list of feature for the input images x extracted using the cnn model.
      features[i] is a PyTorch Tensor of shape (N, C_i, H_i, W_i); recall that features
      from different layers of the network may have different numbers of channels (C_i) and
      spatial dimensions (H_i, W_i).
    """
    features = []
    prev_feat = x
    for i, module in enumerate(cnn._modules.values()):
        next_feat = module(prev_feat)
        features.append(next_feat)
        prev_feat = next_feat
    return features

#please disregard warnings about initialization

/home/por1329/anaconda3/envs/gaussian_splatting/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/por1329/anaconda3/envs/gaussian_splatting/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /home/por1329/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth


  0%|          | 0.00/4.73M [00:00<?, ?B/s]

## Computing Loss

이제 손실 함수의 세 가지 구성 요소를 계산할 것입니다.<br>

손실 함수는 세 가지 항의 가중합으로 이루어진다:<br> 콘텐츠 손실 + 스타일 손실 + 총변동 손실.<br>

아래에서 이러한 가중 항들을 계산하는 함수를 채워 넣게 될 것이다.

## Content loss
우리는 손실 함수에 두 이미지를 모두 포함시켜, 한 이미지의 콘텐츠와 다른 이미지의 스타일을 반영하는 이미지를 생성할 수 있습니다.<br>
목표는 콘텐츠 이미지의 콘텐츠에서 벗어나는 정도와 스타일 이미지의 스타일에서 벗어나는 정도를 모두 패널티로 주는 것입니다.<br>
이후 이 혼합 손실 함수를 사용하여 모델 파라미터가 아닌, 초기 생성 이미지의 픽셀 값에 대해 경사하강을 수행합니다.<br>

먼저 콘텐츠 손실 함수를 정의합니다.<br>
콘텐츠 손실은 생성된 이미지의 특징 맵(feature map)이 원본 콘텐츠 이미지의 특징 맵과 얼마나 다른지를 측정합니다.<br>
네트워크의 특정 한 레이어(예: 레이어 $\ell$)의 콘텐츠 표현만 고려하며, 이 레이어의 특징 맵은 다음과 같습니다:

$A^\ell \in \mathbb{R}^{1 \times C_\ell \times H_\ell \times W_\ell}$

$C_\ell$: 레이어 $\ell$의 필터(채널) 수

$H_\ell, W_\ell$: 특징 맵의 높이와 너비

이제 공간적 위치를 하나의 차원으로 합친 리쉐이프된 특징 맵을 이용한다. 현재 생성 이미지의 특징 맵은

$F^\ell \in \mathbb{R}^{C_\ell \times M_\ell}$

콘텐츠 원본 이미지의 특징 맵은

$P^\ell \in \mathbb{R}^{C_\ell \times M_\ell}$

여기서 $M_\ell = H_\ell \times W_\ell$ 는 각 특징 맵의 요소 개수입니다.
$F^\ell$ 또는 $P^\ell$의 각 행은 특정 필터가 이미지 전 위치에 대해 활성화된 값을 벡터로 펼친 것입니다.
또한, 손실 함수에서 콘텐츠 손실 항의 가중치를 $w_c$라 하자.

콘텐츠 손실은 다음과 같이 정의된다:
$L_c = w_c \times \sum_{i,j} (F_{ij}^{\ell} - P_{ij}^{\ell})^2$

## Assignment 1

아래의 `Content loss`의 함수를 구현하시오

In [ ]:
def content_loss(content_weight, content_current, content_original):
    """
    Compute the content loss for style transfer.
    
    Inputs:
    - content_weight: Scalar giving the weighting for the content loss.
    - content_current: features of the current image; this is a PyTorch Tensor of shape
      (1, C_l, H_l, W_l).
    - content_target: features of the content image, Tensor with shape (1, C_l, H_l, W_l).
    
    Returns:
    - scalar content loss
    """
    pass


content loss를 테스트하시오.<br>
오류는 0.0001보다 작게 나타나야 합니다.

In [ ]:
def content_loss_test(correct):
    content_image = 'styles/tubingen.jpg'
    image_size =  192
    content_layer = 3
    content_weight = 6e-2
    
    c_feats, content_img_var = features_from_img(content_image, image_size)
    
    bad_img = torch.zeros(*content_img_var.data.size()).type(dtype)
    feats = extract_features(bad_img, cnn)
    
    student_output = content_loss(content_weight, c_feats[content_layer], feats[content_layer]).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Maximum error is {:.3f}'.format(error))

content_loss_test(answers['cl_out'])

## Style loss
이제 스타일 손실을 정의한다.<br>
특정 레이어 $\ell$에 대해 스타일 손실은 다음과 같이 정의된다.<br>

먼저 해당 레이어의 특징 맵 $F$(앞서 정의한 형식과 동일)를 사용하여 Gram 행렬 $G$를 계산한다.<br>
Gram 행렬은 각 필터의 반응 간 상관 관계를 나타내며, 이는 공분산 행렬의 근사값이다. 생성 이미지의 활성화 통계가 스타일 이미지의 활성화 통계와 일치하도록 만들고 싶기 때문에, 공분산(또는 그 근사치)을 맞추는 방식이 적합하다.<br>
구현 방식은 여러 가지가 있을 수 있지만 Gram 행렬은 계산이 간단하고 실제로 우수한 결과를 보여 널리 사용된다.

특징 맵 $F^\ell$의 형태가 $(C_\ell, M_\ell)$일 때, Gram 행렬의 형태는 $(C_\ell, C_\ell)$이며, 원소는 다음과 같다:
$$G_{ij}^\ell  = \sum_k F^{\ell}_{ik} F^{\ell}_{jk}$$

여기서

$G^\ell$: 현재 이미지의 Gram 행렬

$A^\ell$: 스타일 이미지의 Gram 행렬

$w_\ell$: 레이어 $\ell$에 대한 가중치

레이어 $\ell$의 스타일 손실은 두 Gram 행렬 간의 가중 유클리드 거리로 정의된다:
$$L_s^\ell = w_\ell \sum_{i, j} \left(G^\ell_{ij} - A^\ell_{ij}\right)^2$$

실제로는 단일 레이어 $\ell$이 아니라 여러 레이어 집합 $\mathcal{L}$에서 스타일 손실을 계산하는 경우가 많으며, 이때 전체 스타일 손실은 각 레이어 스타일 손실의 합이다:

$$L_s = \sum_{\ell \in \mathcal{L}} L_s^\ell$$

## Assignment 2
아래에서 Gram 행렬 계산을 구현하시오.

In [ ]:
def gram_matrix(features, normalize=True):
    """
    Compute the Gram matrix from features.
    
    Inputs:
    - features: PyTorch Tensor of shape (N, C, H, W) giving features for
      a batch of N images.
    - normalize: optional, whether to normalize the Gram matrix
        If True, divide the Gram matrix by the number of neurons (H * W * C)
    
    Returns:
    - gram: PyTorch Tensor of shape (N, C, C) giving the
      (optionally normalized) Gram matrices for the N input images.
    """
    pass


Gram 행렬 코드를 테스트하시오. 오차는 0.0001보다 작아야 합니다.

In [ ]:
def gram_matrix_test(correct):
    style_image = 'styles/starry_night.jpg'
    style_size = 192
    feats, _ = features_from_img(style_image, style_size)
    student_output = gram_matrix(feats[5].clone()).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Maximum error is {:.3f}'.format(error))
    
gram_matrix_test(answers['gm_out'])

## Assignment 3
다음으로, 스타일 손실을 구현하시오.

In [ ]:
# Now put it together in the style_loss function...
def style_loss(feats, style_layers, style_targets, style_weights):
    """
    Computes the style loss at a set of layers.
    
    Inputs:
    - feats: list of the features at every layer of the current image, as produced by
      the extract_features function.
    - style_layers: List of layer indices into feats giving the layers to include in the
      style loss.
    - style_targets: List of the same length as style_layers, where style_targets[i] is
      a PyTorch Tensor giving the Gram matrix of the source style image computed at
      layer style_layers[i].
    - style_weights: List of the same length as style_layers, where style_weights[i]
      is a scalar giving the weight for the style loss at layer style_layers[i].
      
    Returns:
    - style_loss: A PyTorch Tensor holding a scalar giving the style loss.
    """
    # Hint: you can do this with one for loop over the style layers, and should
    # not be very much code (~5 lines). You will need to use your gram_matrix function.
    pass


스타일 손실 구현을 테스트하시오. 오차는 0.0001보다 작아야 합니다.

In [ ]:
def style_loss_test(correct):
    content_image = 'styles/tubingen.jpg'
    style_image = 'styles/starry_night.jpg'
    image_size =  192
    style_size = 192
    style_layers = [1, 4, 6, 7]
    style_weights = [300000, 1000, 15, 3]
    
    c_feats, _ = features_from_img(content_image, image_size)    
    feats, _ = features_from_img(style_image, style_size)
    style_targets = []
    for idx in style_layers:
        style_targets.append(gram_matrix(feats[idx].clone()))
    
    student_output = style_loss(c_feats, style_layers, style_targets, style_weights).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Error is {:.3f}'.format(error))

    
style_loss_test(answers['sl_out'])

## Total-variation regularization
이미지에 매끄러움(smoothness)을 유도하는 것이 도움이 된다는 사실이 알려져 있다. 이를 위해, 픽셀 값의 흔들림(wiggles) 또는 “총변동(total variation)”을 패널티로 주는 항을 손실 함수에 추가할 수 있다.

“총변동(total variation)”은 서로 인접한 모든 픽셀 쌍(수평 또는 수직 방향)에 대해, 픽셀 값의 차이를 제곱하여 모두 더한 값으로 계산한다. 여기서는 RGB 3개 입력 채널 각각에 대해 총변동 정규화를 계산한 뒤, 이를 모두 합산하고 총변동 가중치 $w_t$ 를 곱하여 최종 TV 손실을 정의한다:

$L_{tv} = w_t \times \left(\sum_{c=1}^3\sum_{i=1}^{H-1}\sum_{j=1}^{W} (x_{i+1,j,c} - x_{i,j,c})^2 + \sum_{c=1}^3\sum_{i=1}^{H}\sum_{j=1}^{W - 1} (x_{i,j+1,c} - x_{i,j,c})^2\right)$

다음 셀에서 TV 손실 항을 정의하라. 전체 점수를 받으려면 구현에 루프를 포함하면 안 된다.

In [ ]:
def tv_loss(img, tv_weight):
    """
    Compute total variation loss.
    
    Inputs:
    - img: PyTorch Variable of shape (1, 3, H, W) holding an input image.
    - tv_weight: Scalar giving the weight w_t to use for the TV loss.
    
    Returns:
    - loss: PyTorch Variable holding a scalar giving the total variation loss
      for img weighted by tv_weight.
    """
    # Your implementation should be vectorized and not require any loops!
    pass


TV 손실 구현을 테스트하시오. 오차는 0.0001보다 작아야 합니다.

In [ ]:
def tv_loss_test(correct):
    content_image = 'styles/tubingen.jpg'
    image_size =  192
    tv_weight = 2e-2

    content_img = preprocess(PIL.Image.open(content_image), size=image_size)
    
    student_output = tv_loss(content_img, tv_weight).cpu().data.numpy()
    error = rel_error(correct, student_output)
    print('Error is {:.3f}'.format(error))
    
tv_loss_test(answers['tv_out'])

이제 모든 것을 하나로 연결할 준비가 되었습니다!! (아래의 함수는 수정할 필요가 없어야 한다).

In [ ]:
def style_transfer(content_image, style_image, image_size, style_size, content_layer, content_weight,
                   style_layers, style_weights, tv_weight, init_random = False):
    """
    Run style transfer!
    
    Inputs:
    - content_image: filename of content image
    - style_image: filename of style image
    - image_size: size of smallest image dimension (used for content loss and generated image)
    - style_size: size of smallest style image dimension
    - content_layer: layer to use for content loss
    - content_weight: weighting on content loss
    - style_layers: list of layers to use for style loss
    - style_weights: list of weights to use for each layer in style_layers
    - tv_weight: weight of total variation regularization term
    - init_random: initialize the starting image to uniform random noise
    """
    
    # Extract features for the content image
    content_img = preprocess(PIL.Image.open(content_image), size=image_size)
    feats = extract_features(content_img, cnn)
    content_target = feats[content_layer].clone()

    # Extract features for the style image
    style_img = preprocess(PIL.Image.open(style_image), size=style_size)
    feats = extract_features(style_img, cnn)
    style_targets = []
    for idx in style_layers:
        style_targets.append(gram_matrix(feats[idx].clone()))

    # Initialize output image to content image or nois
    if init_random:
        img = torch.Tensor(content_img.size()).uniform_(0, 1).type(dtype)
    else:
        img = content_img.clone().type(dtype)

    # We do want the gradient computed on our image!
    img.requires_grad_()
    
    # Set up optimization hyperparameters
    initial_lr = 3.0
    decayed_lr = 0.1
    decay_lr_at = 180

    # Note that we are optimizing the pixel values of the image by passing
    # in the img Torch tensor, whose requires_grad flag is set to True
    optimizer = torch.optim.Adam([img], lr=initial_lr)
    
    f, axarr = plt.subplots(1,2)
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[0].set_title('Content Source Img.')
    axarr[1].set_title('Style Source Img.')
    axarr[0].imshow(deprocess(content_img.cpu()))
    axarr[1].imshow(deprocess(style_img.cpu()))
    plt.show()
    plt.figure()
    
    for t in range(200):
        if t < 190:
            img.data.clamp_(-1.5, 1.5)
        optimizer.zero_grad()

        feats = extract_features(img, cnn)
        
        # Compute loss
        c_loss = content_loss(content_weight, feats[content_layer], content_target)
        s_loss = style_loss(feats, style_layers, style_targets, style_weights)
        t_loss = tv_loss(img, tv_weight) 
        loss = c_loss + s_loss + t_loss
        
        loss.backward()

        # Perform gradient descents on our image values
        if t == decay_lr_at:
            optimizer = torch.optim.Adam([img], lr=decayed_lr)
        optimizer.step()

        if t % 100 == 0:
            print('Iteration {}'.format(t))
            plt.axis('off')
            plt.imshow(deprocess(img.data.cpu()))
            plt.show()
    print('Iteration {}'.format(t))
    plt.axis('off')
    plt.imshow(deprocess(img.data.cpu()))
    plt.show()

# 멋진 이미지들을 만들어보자!

아래의 세 가지 서로 다른 파라미터 세트에 대해 style_transfer를 실행해보라. 세 개의 셀을 모두 실행해야 한다. 자유롭게 파라미터를 추가해도 되지만, 제출하는 노트북에는 세 번째 파라미터 세트(starry night) 에 대한 스타일 트랜스퍼 결과를 반드시 포함해야 한다.

파라미터 설명:

* content_image: 콘텐츠 이미지의 파일 이름

* style_image: 스타일 이미지의 파일 이름

* image_size: 콘텐츠 이미지의 가장 작은 차원의 크기

        콘텐츠 손실 및 생성 이미지에 사용됨

* style_size: 스타일 이미지의 가장 작은 차원의 크기

* content_layer: 콘텐츠 손실에 사용할 레이어

* content_weight: 콘텐츠 손실 가중치

        값이 클수록 결과 이미지가 콘텐츠 이미지와 더 유사해짐

* style_layers: 스타일 손실에 사용할 레이어들의 리스트

* style_weights: 각 스타일 레이어에 대응하는 가중치 목록

        보통 더 이른 레이어에 더 높은 가중치를 부여

        값을 키우면 스타일 이미지의 영향을 크게 받아 왜곡이 증가함

* tv_weight: 총변동 정규화의 가중치

        값을 키우면 결과 이미지가 더 매끄럽지만, 콘텐츠·스타일 정확도는 떨어짐

### 실험 안내

다음 세 개의 코드 셀은 하이퍼파라미터를 수정하지 말고 그대로 실행해야 한다.
그 아래에서 파라미터를 복사하여 자유롭게 실험해보고, 결과 이미지가 어떻게 변하는지 확인해도 된다.

In [ ]:
# Composition VII + Tubingen
params1 = {
    'content_image' : 'styles/tubingen.jpg',
    'style_image' : 'styles/composition_vii.jpg',
    'image_size' : 192,
    'style_size' : 512,
    'content_layer' : 3,
    'content_weight' : 5e-2, 
    'style_layers' : (1, 4, 6, 7),
    'style_weights' : (20000, 500, 12, 1),
    'tv_weight' : 5e-2
}

style_transfer(**params1)

In [ ]:
# Scream + Tubingen
params2 = {
    'content_image':'styles/tubingen.jpg',
    'style_image':'styles/the_scream.jpg',
    'image_size':192,
    'style_size':224,
    'content_layer':3,
    'content_weight':3e-2,
    'style_layers':[1, 4, 6, 7],
    'style_weights':[200000, 800, 12, 1],
    'tv_weight':2e-2
}

style_transfer(**params2)

In [ ]:
# Starry Night + Tubingen
params3 = {
    'content_image' : 'styles/tubingen.jpg',
    'style_image' : 'styles/starry_night.jpg',
    'image_size' : 192,
    'style_size' : 192,
    'content_layer' : 3,
    'content_weight' : 6e-2,
    'style_layers' : [1, 4, 6, 7],
    'style_weights' : [300000, 1000, 15, 3],
    'tv_weight' : 2e-2
}

style_transfer(**params3)

## Feature Inversion

당신이 작성한 코드는 또 하나의 흥미로운 작업을 수행할 수 있다. 합성곱 신경망이 어떤 종류의 특징(feature)을 학습하는지 이해하기 위한 시도로, 최근 연구 [1]에서는 특징 표현(feature representation)으로부터 원본 이미지를 재구성하는 방법을 제안했다. 이는 사전학습된 네트워크로부터 이미지 그래디언트를 사용하는 것으로 구현할 수 있으며, 이는 우리가 위에서 수행한 방식과 동일하다(단지 두 종류의 특징 표현을 동시에 사용하지 않았을 뿐이다).

이제 스타일 가중치를 모두 0으로 설정하고, 시작 이미지를 콘텐츠 원본 이미지 대신 무작위 노이즈(random noise)로 초기화하면, 콘텐츠 이미지의 특징 표현으로부터 이미지를 재구성할 수 있다. 초기에는 완전한 노이즈이지만, 최종적으로는 원본 이미지와 상당히 유사한 이미지가 얻어진다.

(비슷하게, 콘텐츠 가중치를 0으로 설정하고 시작 이미지를 무작위 노이즈로 초기화하면, “텍스처 합성(texture synthesis)”을 처음부터 수행할 수도 있다. 하지만 여기서는 그것을 요구하지는 않는다.)

아래 셀을 실행하여 feature inversion을 시도해보라.

[1] Aravindh Mahendran, Andrea Vedaldi, Understanding Deep Image Representations by Inverting them, CVPR 2015

In [ ]:
# Feature Inversion -- Starry Night + Tubingen
params_inv = {
    'content_image' : 'styles/tubingen.jpg',
    'style_image' : 'styles/starry_night.jpg',
    'image_size' : 192,
    'style_size' : 192,
    'content_layer' : 3,
    'content_weight' : 6e-2,
    'style_layers' : [1, 4, 6, 7],
    'style_weights' : [0, 0, 0, 0], # we discard any contributions from style to the loss
    'tv_weight' : 2e-2,
    'init_random': True # we want to initialize our image to be random
}

style_transfer(**params_inv)